In [63]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [64]:
train_data = pd.read_csv('/content/train_2kmZucJ.csv',)
train_data = train_data.reset_index(drop = True)
test_data = pd.read_csv('/content/test_oJQbWVk.csv')
test_data = test_data.reset_index(drop = True)

In [65]:
train_data.shape

(7920, 3)

In [27]:
test_data.shape

(1953, 2)

In [66]:
train_data.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [56]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [67]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

#Sad emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emoticons = emoticons_happy.union(emoticons_sad)

!pip install tweet-preprocessor

In [68]:
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [77]:
from nltk.tokenize import sent_tokenize, word_tokenize
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []#looping through conditions
    for w in word_tokens:
      #check tokens against stop words , emoticons and punctuations
      if w not in stop_words and w not in emoticons and w not in string.punctuation:
        filtered_tweet.append(w)
    return ' '.join(filtered_tweet)

round1 = lambda x: clean_tweets(x)   

In [82]:
def remove_urls(vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

round2 = lambda x: remove_urls(x) 

def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round3 = lambda x: clean_text_round2(x)

In [81]:
data_clean = pd.DataFrame(train_data.tweet.apply(round2))
data_clean = pd.DataFrame(data_clean.tweet.apply(round1))
data_clean.tweet[]

'We love Would go talk makememories unplug relax iphone smartphone wifi connect ...'

In [84]:
data_clean =  pd.DataFrame(data_clean.tweet.apply(round3))
# data_clean.tweet[]

In [87]:
data_clean.tweet[2]

'We love Would go talk makememories unplug relax iphone smartphone wifi connect ...'

In [37]:
def clean_round_1(text):
  corpus = []
  text = re.sub(r'[^a-zA-Z]', ' ', data['tweet'][i])
  text = text.lower()
  text = text.split()
  """Applying Steming"""
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  ps = PorterStemmer() #loved --> love word thast essentially means the same
  text = [ps.stem(word) for word in text if not word in set(all_stopwords)]
  corpus = ' '.join(text)
  return corpus

round1 = lambda x: clean_round_1(x)

def clean_round_2(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text) 

round1 = lambda x: clean_round_2(x)


In [16]:
print(corp)

['seri escapad demonstr adag good goos also good gander occasion amus none amount much stori', 'seri escapad demonstr adag good goos', 'seri', '', 'seri', 'escapad demonstr adag good goos', '', 'escapad demonstr adag good goos', 'escapad', 'demonstr adag good goos', 'demonstr adag', 'demonstr', 'adag', '', 'adag', 'good goos', '', 'good goos', '', 'good goos', '', 'good goos', 'good', 'goos', '', 'goos', 'goos', 'also good gander occasion amus none amount much stori', 'also good gander occasion amus none amount much stori', 'also', 'also', 'good gander occasion amus none amount much stori', 'gander occasion amus none amount much stori', 'gander occasion amus none amount much stori', 'gander', 'gander', 'gander', '', 'occasion amus none amount much stori', '', '', '', '', 'occasion amus none amount much stori', 'occasion', 'amus none amount much stori', 'amus', 'none amount much stori', '', 'none amount much stori', 'none', 'amount much stori', 'amount much stori', 'amount much stori', 